# emoji prediction

use the column 'sentence', choose a model to get predicted emojis from this pure sentence,

create two new columns:   
'predicted_emojis' # get 3 predicted emojis from sentence    
'predicted_sentence' # combine the predicted emojis with the original pure sentence



In [76]:
import pandas as pd
import re

import os
from together import Together

from openai import OpenAI
import random
from collections import Counter
import time

import math

In [77]:
prompt_augmentation = False

In [78]:
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
model_name = 'gpt-4o'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
# model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

gpt-4o
gpt-4o-mini


if you wanna prompt augmentation, set the parameter 'prompt_augmentation=True',   
prompt augmentation means using more detailed prompt, this prompt contains the classification from sentence.

## read csv to get dataset dataframe

In [37]:
# !ls ./dataset_predicted/{model_name}/prompt_aug_sentence_cls
!ls ./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_5039.csv


./dataset_finalOutput/gpt-4o/dataset_finalOutput_gpt-4o_gpt-4o-mini_5039.csv


In [38]:
dataset_path = f'./dataset_finalOutput/{model_name}/dataset_finalOutput_{model_name}_{judge_model_name}_5039.csv'

# dataset_path = f'./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_prompt_aug_sentence_cls_400.csv'

emoji_dataset = pd.read_csv(dataset_path)
dataset_len=len(emoji_dataset)
print(dataset_len)

5039


In [39]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5039 non-null   int64 
 1   index               5039 non-null   int64 
 2   user_id             5039 non-null   object
 3   emoji_sentence      5039 non-null   object
 4   char_count          5039 non-null   int64 
 5   cluster             5039 non-null   int64 
 6   sentence            5039 non-null   object
 7   gender_labels       5039 non-null   object
 8   sentiment_labels    5039 non-null   object
 9   emotion_labels      5039 non-null   object
 10  age_labels          5039 non-null   object
 11  stance_labels       5039 non-null   object
 12  is_duplicate        5039 non-null   bool  
 13  predicted_emojis    5039 non-null   object
 14  predicted_sentence  5039 non-null   object
 15  gender_logits       5039 non-null   object
 16  sentiment_logits    5039

## if prompt_augmentation==True

In [40]:
!ls ./dataset_finalOutput/random_sentence/dataset_labels_using_random_sentence_stance_5039.csv

./dataset_finalOutput/random_sentence/dataset_labels_using_random_sentence_stance_5039.csv


In [41]:
path = './dataset_finalOutput/random_sentence/dataset_labels_using_random_sentence_stance_5039.csv'

emoji_dataset_02 = pd.read_csv(path)
emoji_dataset_02.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 24 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   Unnamed: 0                              5039 non-null   int64 
 1   index                                   5039 non-null   int64 
 2   user_id                                 5039 non-null   object
 3   emoji_sentence                          5039 non-null   object
 4   char_count                              5039 non-null   int64 
 5   cluster                                 5039 non-null   int64 
 6   sentence                                5039 non-null   object
 7   gender_labels                           5039 non-null   object
 8   sentiment_labels                        5039 non-null   object
 9   emotion_labels                          5039 non-null   object
 10  age_labels                              5039 non-null   object
 11  stan

In [42]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          5039 non-null   int64 
 1   index               5039 non-null   int64 
 2   user_id             5039 non-null   object
 3   emoji_sentence      5039 non-null   object
 4   char_count          5039 non-null   int64 
 5   cluster             5039 non-null   int64 
 6   sentence            5039 non-null   object
 7   gender_labels       5039 non-null   object
 8   sentiment_labels    5039 non-null   object
 9   emotion_labels      5039 non-null   object
 10  age_labels          5039 non-null   object
 11  stance_labels       5039 non-null   object
 12  is_duplicate        5039 non-null   bool  
 13  predicted_emojis    5039 non-null   object
 14  predicted_sentence  5039 non-null   object
 15  gender_logits       5039 non-null   object
 16  sentiment_logits    5039

In [43]:
print(emoji_dataset.columns.tolist())
task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
for task in task_list: 
    column_name= f'{task}_labels_using_sentence'
    if column_name not in emoji_dataset.columns.tolist():
        emoji_dataset[column_name] = emoji_dataset_02[column_name]
    

['Unnamed: 0', 'index', 'user_id', 'emoji_sentence', 'char_count', 'cluster', 'sentence', 'gender_labels', 'sentiment_labels', 'emotion_labels', 'age_labels', 'stance_labels', 'is_duplicate', 'predicted_emojis', 'predicted_sentence', 'gender_logits', 'sentiment_logits', 'emotion_logits', 'age_logits', 'stance_logits', 'real_emojis']


In [44]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 26 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Unnamed: 0                       5039 non-null   int64 
 1   index                            5039 non-null   int64 
 2   user_id                          5039 non-null   object
 3   emoji_sentence                   5039 non-null   object
 4   char_count                       5039 non-null   int64 
 5   cluster                          5039 non-null   int64 
 6   sentence                         5039 non-null   object
 7   gender_labels                    5039 non-null   object
 8   sentiment_labels                 5039 non-null   object
 9   emotion_labels                   5039 non-null   object
 10  age_labels                       5039 non-null   object
 11  stance_labels                    5039 non-null   object
 12  is_duplicate                     5

## prediction model is GPT series

In [45]:
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
model_name = 'gpt-4o'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
# model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

gpt-4o
gpt-4o-mini


In [46]:
print(model_name)
print(judge_model_name)

print(dataset_len)

prompt_augmentation=True
print(prompt_augmentation)


gpt-4o
gpt-4o-mini
5039
True


In [ ]:

def predict_emoji_openai(model_name, text, cls_dict=None):
    if prompt_augmentation==True: 
        prompt = f"""
        As a social media user, given the context: "{text}", select three emojis that best represent this context based on the following classifications:
        - Gender: {cls_dict['gender']}
        - Sentiment: {cls_dict['sentiment']}
        - Emotion: {cls_dict['emotion']}
        - Age: {cls_dict['age']}
        - Stance: {cls_dict['stance']}
        Provide only the three emojis that best fit the context, without any additional words or explanations.
        """
    else:
        prompt = f"As a social media user, find three emojis that best fit the context: {text}. Do not include any words or explanations—only three emojis."# Do not include any words or explanations—only three emojis.
    
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
        
    )
    prediction = response.choices[0].message.content.strip()
    return prediction

In [ ]:

api_key= '' # input your openai api key here

client = OpenAI(api_key=bruce_api_key)

In [54]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 28 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

In [56]:


task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
cls_dict={}

start_time = time.time()  
logout_len = 100

if prompt_augmentation==True: 
    folder_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls"
else: 
    folder_path = f"./dataset_predicted/{model_name}"
os.makedirs(folder_path, exist_ok=True)

for i in range(3969, dataset_len):
    text = emoji_dataset['sentence'][i]

    if prompt_augmentation==True: 
        for task in task_list: 
            cls_dict[task] = emoji_dataset.at[i, f'{task}_labels_using_sentence']
        print(cls_dict)
        emojis = predict_emoji_together(model_name_together, text, cls_dict)
        emoji_dataset.at[i, 'predicted_emojis_prompt_aug_sentence_cls'] = emojis
        emoji_dataset.at[i, 'predicted_sentence_prompt_aug_sentence_cls']= emoji_dataset['sentence'][i] + ' ' + emojis

        # folder_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls"
        # os.makedirs(folder_path, exist_ok=True)
    else: 
        emojis = predict_emoji_together(model_name_together, text)
        emoji_dataset.at[i, 'predicted_emojis'] = emojis
        emoji_dataset.at[i, 'predicted_sentence']= emoji_dataset['sentence'][i] + ' ' + emojis

        # folder_path = f"./dataset_predicted/{model_name}"
        # os.makedirs(folder_path, exist_ok=True)
    
    print(i, emojis)
    if (i+1) % 20 == 0: time.sleep(5)
        
    if (i + 1) % logout_len == 0: 
        end_time = time.time()  
        elapsed_time = end_time - start_time  
        print(f"run time of {logout_len} iterations: {elapsed_time:.4f} seconds.")
        
        if prompt_augmentation==True: 
            output_prediction_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_{i+1}.csv"
        else:
            output_prediction_path = f"./dataset_predicted/{model_name}/dataset_predicted_{model_name}_{i+1}.csv"
        # output_file = f"unlabel_data_with_predictions_{i+1}.csv"
        emoji_dataset.to_csv(output_prediction_path, index=False)
        print(f"success saved: {output_prediction_path}")

if prompt_augmentation==True: 
    output_prediction_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_{dataset_len}.csv"
else:
    output_prediction_path = f"./dataset_predicted/{model_name}/dataset_predicted_{model_name}_{dataset_len}.csv"
emoji_dataset.to_csv(output_prediction_path, index=False)
print(f"success saved: {output_prediction_path}")



{'gender': 'male', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'adult', 'stance': 'none'}
3969 🍻👨‍🦰😊
{'gender': 'male', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'adult', 'stance': 'none'}
3970 👨‍💼😃👍
{'gender': 'male', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'adult', 'stance': 'favor'}
3971 💰📈😊
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'happiness', 'age': 'teen', 'stance': 'favor'}
3972 💥⚽️😄
{'gender': 'male', 'sentiment': 'negative', 'emotion': 'sadness', 'age': 'adult', 'stance': 'none'}
3973 😔😢😭
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'surprise', 'age': 'adult', 'stance': 'none'}
3974 Descending man, surprised face, globe showing Australia and Antarctica. 

(👨‍.Down, 😮, 🌍🇦🇶🇦🇺) 

Note: The first emoji is not officially recognized by Unicode, but it is included here to represent a man descending to the south.
{'gender': 'female', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'adult', 'stance': 'favor'}
3975 

## prediction model is other open source model


In [7]:
# model_name='llama3.1-70b-instruct'
# model_name = 'llama3.1-8b-instruct'
# model_name = 'gpt-4o'
# model_name = 'gemma2-9b-instruct'
# model_name= 'mistral-7b-instruct'
model_name = 'qwen2-72b-instruct'

# model_name = 'sentence'
# model_name = 'random_sentence'

judge_model_name = "gpt-4o-mini"
# judge_model_name = "gpt-4o-turbo"

print(model_name)
print(judge_model_name)

qwen2-72b-instruct
gpt-4o-mini


In [8]:
def get_model_name(model_name):
    model_name_together = ''
    if model_name == 'llama3.1-8b-instruct': 
        model_name_together = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo"
    if model_name=='llama3.1-70b-instruct':
        model_name_together = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
    if model_name == 'gemma2-9b-instruct':
        model_name_together = 'google/gemma-2-9b-it'
    if model_name == 'mistral-7b-instruct':
        model_name_together = 'mistralai/Mistral-7B-Instruct-v0.3'
    if model_name == 'qwen2-72b-instruct':
        model_name_together = 'Qwen/Qwen2-72B-Instruct'
    return model_name_together

In [9]:
print(model_name)
print(judge_model_name)

print(dataset_len)

prompt_augmentation=True
print(prompt_augmentation)


qwen2-72b-instruct
gpt-4o-mini
5039
True


In [10]:


def predict_emoji_together(model_name_together, text, cls_dict=None):
    
    if prompt_augmentation==True: 
        prompt = f"""
        As a social media user, given the context: "{text}", select three emojis that best represent this context based on the following classifications:
        - Gender: {cls_dict['gender']}
        - Sentiment: {cls_dict['sentiment']}
        - Emotion: {cls_dict['emotion']}
        - Age: {cls_dict['age']}
        - Stance: {cls_dict['stance']}
        Provide only the three emojis that best fit the context, without any additional words or explanations.
        """
    else:
        prompt = f"As a social media user, find three emojis that best fit the context: {text}. Do not include any words or explanations—only three emojis."# Do not include any words or explanations—only three emojis.
    
    response = client.chat.completions.create(
        model=model_name_together, # meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
        messages=[
            {
                "role": "user",
                "content": prompt
            },
        ],
    )
    prediction = response.choices[0].message.content.strip()
    return prediction
    


In [11]:
# predicted_emojis_list = []
model_name_together = get_model_name(model_name)


In [25]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 30 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

In [31]:



together_api_token= '' # input your together api key here # https://api.together.xyz/

client = Together(api_key=together_api_token)

task_list = ['gender', 'sentiment', 'emotion', 'age', 'stance']
cls_dict={}

# prompt aug
# qwen 61 535 3929
start_time = time.time() 
logout_len = 100

if prompt_augmentation==True: 
    folder_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls"
else: 
    folder_path = f"./dataset_predicted/{model_name}"
os.makedirs(folder_path, exist_ok=True)

for i in range(4318, dataset_len):
    text = emoji_dataset['sentence'][i]

    if prompt_augmentation==True: 
        for task in task_list: 
            cls_dict[task] = emoji_dataset.at[i, f'{task}_labels_using_sentence']
        print(cls_dict)
        emojis = predict_emoji_together(model_name_together, text, cls_dict)
        emoji_dataset.at[i, 'predicted_emojis_prompt_aug_sentence_cls'] = emojis
        emoji_dataset.at[i, 'predicted_sentence_prompt_aug_sentence_cls']= emoji_dataset['sentence'][i] + ' ' + emojis

        # folder_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls"
        # os.makedirs(folder_path, exist_ok=True)
    else: 
        emojis = predict_emoji_together(model_name_together, text)
        emoji_dataset.at[i, 'predicted_emojis'] = emojis
        emoji_dataset.at[i, 'predicted_sentence']= emoji_dataset['sentence'][i] + ' ' + emojis

        # folder_path = f"./dataset_predicted/{model_name}"
        # os.makedirs(folder_path, exist_ok=True)
    
    print(i, emojis)
    if (i+1) % logout_len == 0: time.sleep(5)
        
    if (i + 1) % logout_len == 0: 
        end_time = time.time()  
        elapsed_time = end_time - start_time  
        print(f"run time of {logout_len} iterations: {elapsed_time:.4f} seconds.")
        
        if prompt_augmentation==True: 
            output_prediction_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_{i+1}.csv"
        else:
            output_prediction_path = f"./dataset_predicted/{model_name}/dataset_predicted_{model_name}_{i+1}.csv"
        # output_file = f"unlabel_data_with_predictions_{i+1}.csv"
        emoji_dataset.to_csv(output_prediction_path, index=False)
        print(f"success saved: {output_prediction_path}")

if prompt_augmentation==True: 
    output_prediction_path = f"./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_{dataset_len}.csv"
else:
    output_prediction_path = f"./dataset_predicted/{model_name}/dataset_predicted_{model_name}_{dataset_len}.csv"
emoji_dataset.to_csv(output_prediction_path, index=False)
print(f"success saved: {output_prediction_path}")



{'gender': 'female', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'child', 'stance': 'none'}
4318 🎉😊🎈
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'surprise', 'age': 'child', 'stance': 'none'}
4319 😲👦 Intercepted BALL⚽️
{'gender': 'male', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'teen', 'stance': 'none'}
4320 😂🤣😆
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'surprise', 'age': 'adult', 'stance': 'none'}
4321 🤔😕🤷‍♂️
{'gender': 'female', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'teen', 'stance': 'none'}
4322 👏🏼💕🎶
{'gender': 'female', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'teen', 'stance': 'none'}
4323 🎉😊👀
{'gender': 'female', 'sentiment': 'positive', 'emotion': 'happiness', 'age': 'adult', 'stance': 'favor'}
4324 ❤️🇺🇸👩🏻‍🦰
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'happiness', 'age': 'teen', 'stance': 'none'}
4325 ⚽️😊👍🏻
{'gender': 'male', 'sentiment': 'neutral', 'emotion': 'happiness', 'age': 'teen', '

In [30]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 30 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

## other: get pure emojis from predicted_emojis

if the prediction model is 'qwen2-72b-instruct' or 'mistral-7b-instruct', the 'predicted_emojis' contains text, we need to extract pure emojis.  

and create two new columns: predicted_pure_emojis, predicted_pure_sentence.   



In [57]:
# model_name = 'mistral-7b-instruct'

model_name = 'qwen2-72b-instruct'

In [58]:
specific_models = ['mistral-7b-instruct', 'qwen2-72b-instruct']

print(model_name)
if model_name not in specific_models: 
    print('it is not specific model, don not need to get predicted_pure_emojis.')

qwen2-72b-instruct


In [60]:
# !ls ./dataset_predicted/dataset_predicted_qwen2-72b-instruct_5039.csv

!ls ./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_5039.csv


./dataset_predicted/qwen2-72b-instruct/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_prompt_aug_sentence_cls_5039.csv


In [63]:
# dataset_path = './dataset_predicted/mistral-7b-instruct/prompt_aug_sentence_cls/dataset_predicted_mistral-7b-instruct_prompt_aug_sentence_cls_5039.csv'
# 

dataset_path = f'./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_prompt_aug_sentence_cls_5039.csv'

In [64]:
emoji_dataset = pd.read_csv(dataset_path)
dataset_len=len(emoji_dataset)
print(dataset_len)

5039


In [65]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 30 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   Unnamed: 0                                  5039 non-null   int64 
 1   index                                       5039 non-null   int64 
 2   user_id                                     5039 non-null   object
 3   emoji_sentence                              5039 non-null   object
 4   char_count                                  5039 non-null   int64 
 5   cluster                                     5039 non-null   int64 
 6   sentence                                    5039 non-null   object
 7   gender_labels                               5039 non-null   object
 8   sentiment_labels                            5039 non-null   object
 9   emotion_labels                              5039 non-null   object
 10  age_labels              

In [66]:
prompt_augmentation = True
print(prompt_augmentation)

True


In [67]:

def extract_emojis(text):
    if text is None or (isinstance(text, float) and math.isnan(text)):
        return [] 
    emoji_pattern = re.compile(
        "["                              
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & map symbols
        u"\U0001F700-\U0001F77F"  # Alchemical symbols
        u"\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
        u"\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA00-\U0001FA6F"  # Chess Symbols
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001FB00-\U0001FBFF"  # Symbols for Legacy Computing
        u"\U0001F004"             # Mahjong Tile Red Dragon
        u"\U0001F0CF"             # Playing Cards Joker
        u"\U0001F1E6-\U0001F1FF"  # Flags
        u"\U000025AA-\U000025AB"  # Small Squares
        u"\U000025FE-\U000025FF"  # Black Medium Squares
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U00002B50"             # Star
        u"\U0001F1E6-\U0001F1FF"  # Regional indicator symbols
        u"\u2640-\u2642"          # Gender signs
        u"\u2600-\u26FF"          # Miscellaneous Symbols
        u"\u2700-\u27BF"          # Dingbats
        "]+", flags=re.UNICODE)

    return emoji_pattern.findall(text)

In [70]:
emojis_list=[]
for i in range(dataset_len):
    if prompt_augmentation==True:
        text=emoji_dataset.at[i,'predicted_emojis_prompt_aug_sentence_cls']
    else:
        text=emoji_dataset.at[i,'predicted_emojis']
    list_emojis = extract_emojis(text)
    # print(i, list_emojis)
    str_emojis=''.join(list_emojis)
    print(i,str_emojis, len(str_emojis))
    emojis_list.append(str_emojis)
    
    if prompt_augmentation==True:
        emoji_dataset.at[i,'predicted_pure_emojis_prompt_aug_sentence_cls']=str_emojis
    else:
        emoji_dataset.at[i,'predicted_pure_emojis']=str_emojis
    # emoji_dataset.at[i, 'predicted_pure_emojis']=str_emojis
    



0 😠😡😤 3
1 👦❗🤔 3
2 😔😢😭 3
3 😠😡🤬 3
4 😡😔💔 3
5 😔😕😠 3
6 😡😠😤 3
7 😡😠😤 3
8 😠💪🚫 3
9 😔😕😢 3
10 😠😡🤯 3
11 👩🦰😠😢 4
12 🌳⚡😱 3
13 😔😢😭 3
14 😠😡😤 3
15 👩🦳 2
16 😔💔😢 3
17 😠😡🤬 3
18 😡😠😤 3
19 😠😡🤬 3
20 😂🙄🤔 3
21 😡😔💔 3
22 🤔📸🦘💪 4
23  0
24  0
25  0
26  0
27  0
28  0
29  0
30  0
31  0
32  0
33  0
34  0
35  0
36  0
37  0
38  0
39  0
40  0
41  0
42  0
43  0
44  0
45  0
46  0
47  0
48  0
49  0
50  0
51  0
52  0
53  0
54  0
55  0
56  0
57  0
58  0
59  0
60  0
61 😡💔🇺🇸 4
62 😡🙏🚫 3
63 ❌🔥😠 3
64 😡🔥💥 3
65 🤯💔😔 3
66 😡🚫 2
67 😱😢😭 3
68 🤢🤮🤯 3
69 🕷🚫😢 3
70 😠😡🤬 3
71 😡❌🇬🇧 4
72 😮🤯😂 3
73 😠😡🚫 3
74 😠😡🤬 3
75 😠😡😤 3
76  0
77 😡😠😤 3
78 😠💰🚫 3
79 😡😔 2
80 🙏🏼🚀 3
81 💩🖍👃 3
82 👩⚖📚 3
83 😠💰❌ 3
84 😢😔😭 3
85 😔😕😖 3
86 😓😣😖 3
87 👎🤢 2
88 😠 1
89 😠💔 2
90 🔥💔😡 3
91 👩⚕🤞😊 4
92 💩😕😣 3
93 😔😕😠 3
94 ❌👎😢 3
95 😠😡🤯 3
96 🥗😖🤷♀ 4
97 😡💔😢 3
98 🤢😂😚🤢 4
99 👦🚫😢 3
100 😡😤🤯 3
101 😔😕😠 3
102 😱💨😴 3
103 😭😢😔 3
104 ❌😣 2
105 😭😢😔 3
106 😡😠😤 3
107 😮😂 2
108 👩🦰❌😖 4
109 ❌💔😡 3
110 😭😢😔 3
111 👏💖😢 3
112 😠😡😤 3
113 😠💔😢 3
114 😠😡🤬 3
115 😱😢💔 3
116 😱😴👀 3
117 😠😔🤷♀ 4
118 😱🙏🔒 3
119 😱😢😱 3
120 💪🚀🎯 3
121 😡😔😔 3
122 😠😔💔

In [71]:

if prompt_augmentation==True:
    emoji_dataset['predicted_pure_sentence_prompt_aug_sentence_cls']=emoji_dataset['sentence']+ ' '+emoji_dataset['predicted_pure_emojis_prompt_aug_sentence_cls']
else:
    emoji_dataset['predicted_pure_sentence']=emoji_dataset['sentence']+ ' '+emoji_dataset['predicted_pure_emojis']



In [72]:
emoji_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5039 entries, 0 to 5038
Data columns (total 32 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Unnamed: 0                                       5039 non-null   int64 
 1   index                                            5039 non-null   int64 
 2   user_id                                          5039 non-null   object
 3   emoji_sentence                                   5039 non-null   object
 4   char_count                                       5039 non-null   int64 
 5   cluster                                          5039 non-null   int64 
 6   sentence                                         5039 non-null   object
 7   gender_labels                                    5039 non-null   object
 8   sentiment_labels                                 5039 non-null   object
 9   emotion_labels                           

In [74]:
!ls ./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_prompt_aug_sentence_cls_5039.csv

./dataset_predicted/qwen2-72b-instruct/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_prompt_aug_sentence_cls_5039.csv


In [75]:
print(model_name)
# output_path = './dataset_with_logits/mistral-7b-instruct/stance/dataset_logits_mistral-7b-instruct_stance_5039.csv'
output_path = f'./dataset_predicted/{model_name}/prompt_aug_sentence_cls/dataset_predicted_{model_name}_with_pure_prompt_aug_sentence_cls_5039.csv'

emoji_dataset.to_csv(output_path)
print(f'saved the file: {output_path}')

qwen2-72b-instruct
saved the file: ./dataset_predicted/qwen2-72b-instruct/prompt_aug_sentence_cls/dataset_predicted_qwen2-72b-instruct_with_pure_prompt_aug_sentence_cls_5039.csv
